In [1]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="org.esa.snap.core.dataio.dimap.DimapProductReader")


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


In [2]:

def check_image(mgrs):
    processed_mosaic = os.listdir("/data/ksa/01_Image_Acquisition/02_Processed_mosaic/")
    if(mgrs in processed_mosaic):
        return True
    else: 
        return False

def get_gc(path_to_sentinel_data):
    # path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    gc = product_subset.getSceneGeoCoding()
    bands_names = list(product_subset.getBandNames())
    return product_subset,gc,bands_names

def get_values(gc, bands_names, lat, long):
    pixel_pos = gc.getPixelPos(GeoPos(lat, long), None)
    data = list()
    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
            data_values = [float(val) for val in data]
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data_values

# data_values
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(band_name)
            print(width, height)
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            traceback.print_exc()
    return data


In [3]:
## 32 35 16 63 
## 12 73 
## 18 53 94 97  

## edit with province
idprov = "73"

In [4]:
df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
# df_ksa.iloc[:1000].to_csv("cek.csv")

# df_ksa = pd.read_csv("/data/raw/processed/generated_points_unstable.csv")
df_ksa['idprov'] = df_ksa.idsegmen.astype('str').str[:2]
df_ksa['idkab'] = df_ksa.idsegmen.astype('str').str[:4]

df_ksa['index'] = [x.zfill(2) for x in df_ksa['index'].astype("str")]
df_ksa['idpoint'] = df_ksa.idsegmen.astype('str') + df_ksa.idsubsegmen.astype('str') + '#' + df_ksa['index']

df_ksa = df_ksa.query('idprov == "{0}"'.format(idprov))

df_ksa.head()

,iterx,itery,lat,long,index,idsegmen,strati,idsubsegmen,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,idprov,idkab,idpoint
5120775,1,1,-7.103411,120.640848,01,730102004,S1,A1,200000mE,9200000mN,TN,51M,51MTN,73,7301,730102004A1#01
5120776,1,2,-7.103590,120.640848,02,730102004,S1,A1,200000mE,9200000mN,TN,51M,51MTN,73,7301,730102004A1#02
5120777,1,3,-7.103768,120.640848,03,730102004,S1,A1,200000mE,9200000mN,TN,51M,51MTN,73,7301,730102004A1#03
5120778,1,4,-7.103946,120.640848,04,730102004,S1,A1,200000mE,9200000mN,TN,51M,51MTN,73,7301,730102004A1#04
5120779,1,5,-7.104125,120.640848,05,730102004,S1,A1,200000mE,9200000mN,TN,51M,51MTN,73,7301,730102004A1#05


In [6]:
## list all MGRS
mgrs_all_ = list(df_ksa.MGRS.unique())

## Cek MGRS not done
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

['51MTN',
 '51MTP',
 '51MSP',
 '51MTQ',
 '51MSQ',
 '50MRU',
 '50MQU',
 '50MQV',
 '50MRV',
 '50MQA',
 '50MRA',
 '51MSR',
 '51MTR',
 '50MQB',
 '50MRB',
 '51MTS',
 '51MSS',
 '51MST',
 '51MTT',
 '51MUT']

In [ ]:
for mgrs in mgrs_not_done_:
    df_result = pd.DataFrame()
    if(check_image(mgrs)):
        try:
            print(mgrs + " available")
            path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/"+mgrs+"/"
            all_sentinel_data = glob(path_to_mgrs+"*.dim")
            df_ksa_mgrs = df_ksa.loc[df_ksa.MGRS == mgrs]
            for sent in tqdm(all_sentinel_data):
                path_to_sentinel_data = sent
                periode = path_to_sentinel_data[-21:-4]
                product_subset,gc,bands_names = get_gc(path_to_sentinel_data)
                bands_names = [col for col in bands_names if 'VH' in col]
                df_ksa_mgrs_tmp = df_ksa_mgrs.copy()
                df_ksa_mgrs_tmp = df_ksa_mgrs_tmp.reset_index(drop=True)
                df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
                df_ksa_mgrs_tmp.loc[:, 'periode'] =periode  
                # print(bands_names)
                for i,r in df_ksa_mgrs_tmp.iterrows():
                    val = get_values(gc, bands_names, r['lat'], r['long'])
                    df_ksa_mgrs_tmp.loc[i,bands_names] = val
                    # break
                # break
                df_result = pd.concat([df_result,df_ksa_mgrs_tmp])
    
            df_result = df_result[['idpoint','MGRS','Sigma0_VH_db','periode']]
            with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_result, f)
            
            print("done sampling for : "+ mgrs)
            # break
        except: 
            print("Error on : "+ mgrs)
    else:
        print(mgrs + " not available")
        continue
    # break

51MTN available


100%|██████████| 60/60 [03:54<00:00,  3.90s/it]


done sampling for : 51MTN
51MTP available


100%|██████████| 60/60 [04:48<00:00,  4.81s/it]


done sampling for : 51MTP
51MSP available


...

100%|██████████| 60/60 [18:37<00:00, 18.63s/it]


done sampling for : 51MTQ
51MSQ available


100%|██████████| 60/60 [1:26:49<00:00, 86.83s/it] 


done sampling for : 51MSQ
50MRU available


  7%|▋         | 4/60 [04:39<1:04:46, 69.41s/it]

In [ ]:
## Cek error mgrs
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

### Cek 

In [ ]:
## Cek VH Value Overtime on One Point

filtered_df = df_.query('idpoint == "321201001A1#02"').sort_values('periode')

plt.figure(figsize=(10, 6))
plt.plot(filtered_df['periode'], filtered_df['Sigma0_VH_db'], marker='o')

plt.title('Sigma0_VH_db over Time for idpoint "321201003B2#13"')
plt.xlabel('Periode')
plt.ylabel('Sigma0_VH_db')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels if needed

plt.tight_layout()
plt.show()

In [ ]:
## Open one Sampling Data

mgrs = '49LDM'
with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
    df_ = pickle.load(f)

df_.head()

In [ ]:
df_.describe()